# Import libraries

In [170]:
import sys
sys.path.append('modules')

# Import Custom Modules
import modules.source as src
import modules.target as tgt

### Setting local varibale

In [171]:
is_debugging = 1

wtb_1_any_ds_url   = "https://finance.yahoo.com/quote/"
wtb_2_any_ds_path  = "NVDA/history/?period1=1709544776&period2=1741080767&filter=history" 
wtb_3_any_ni_index = "0"

nm_target_schema = "tsl_psa_yahoo_exchange_rate"
nm_target_table  = "tsl_eur_x_usd"
 

## Extract web table from Yahoo Finance

In [179]:
# Import Custom Modules
from modules import session     as ss
from modules import secret      as sc 

# Import for web_table_anonymous_web
import pandas as pd
import time
from selenium.webdriver.common.by import By as by
from selenium                     import webdriver
from bs4                          import BeautifulSoup
from io                           import StringIO

In [173]:
# Initialize the WebDriver (e.g., Chrome)
driver = webdriver.Chrome()

# Open the webpage
driver.get(wtb_1_any_ds_url + wtb_2_any_ds_path)

# Wait for the page to load (you might need to adjust the sleep time)
time.sleep(2)

try: # Find and click the "Accept Cookies" button (adjust the selector as needed)
    accept_button = driver.find_element(by.XPATH, '//button[text()="Alles accepteren"]')
    accept_button.click()

    # Wait for the page to load after accepting cookies
    time.sleep(2)

except Exception as e:
    # Code to handle any other exceptions
    print(f"An unexpected error occurred: {e}")        

# Get the page source after accepting cookies
page_source = driver.page_source

# Close the browser
driver.quit()

In [174]:
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Create a StringIO object
table = StringIO()

# Find the table in the webpage (you might need to adjust the selector based on the webpage structure)
table.write(str(soup.find('table')))

# Read the table into a pandas DataFrame
pandas_df = pd.read_html(table.getvalue())[int(wtb_3_any_ni_index)]


C:\Users\mehme\AppData\Local\Temp\ipykernel_11128\2596355597.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pandas_df = pd.read_html(table.getvalue())[int(wtb_3_any_ni_index)]


In [175]:
# Initialize Spark session
spark = ss.getSparkSession("WebTableToSparkDataFrame")

In [176]:
# Convert the pandas DataFrame to a Spark DataFrame
df_source_dataset = spark.createDataFrame(pandas_df)


### Spark DataFrame van Web table avialable

In [180]:
# Import Custom Modules
from modules import credentials as sa


In [182]:
print(ss.jdbc_url(sa.target_db))

AttributeError: module 'modules.session' has no attribute 'jdbc_url'

In [181]:
# Truncate Target Table
ss.truncate_table(sa.target_db, nm_target_schema, nm_target_table)

AttributeError: module 'modules.session' has no attribute 'truncate_table'

In [ ]:
# Database credentials
ds_jdbc_url = ss.jdbc_url(sa.secret_db)
nm_username = sa.target_db['username']
cd_password = sa.target_db['password']

# Load Source DataFrame to SQL Schema / Table
result = df_source_dataset.write.format("jdbc").option("url", ds_jdbc_url).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
    .option("user",     nm_username)\
    .option("password", cd_password)\
    .option("schema",   nm_target_schema) \
    .option("dbtable",  nm_target_table) \
    .mode("append").save()

# Show Input Parameter(s)
if (is_debugging == "1"):
    print(f"nm_target_schema : '{nm_target_schema}'")
    print(f"nm_target_table  : '{nm_target_table}'")
    print(f"ni_ingested      : # {str(result)}")

In [ ]:
# If is Debugging then show imput parameters
if (is_debugging == 1):
    print("wtb_1_any_ds_url   : '" + wtb_1_any_ds_url + "'")
    print("wtb_2_any_ds_path  : '" + wtb_2_any_ds_path + "'")
    print("wtb_3_any_ni_index : '" + wtb_3_any_ni_index + "'")
    print("DataFrame:")
    df.show(10)